# Types

In [1]:
x = 2
typeof(x)

x += 2im
typeof(x)

Complex{Int64}

## Rebinding

In [2]:
x = Int64(2)
x = "cat"

"cat"

## Multiple Dispatch

In [3]:
x, y = 1.0, 1.0
@which +(x, y)

+(x::Float64, y::Float64) in Base at float.jl:375

In [4]:
x, y = 1, 1.0
@which +(x, y)

+(x::Number, y::Number) in Base at promotion.jl:249

### Methods

In [5]:
isfinite(1 / 0)
methods(isfinite)

# 9 methods for generic function "isfinite":
isfinite(x::BigFloat) in Base.MPFR at mpfr.jl:831
isfinite(x::Float16) in Base at float.jl:539
isfinite(x::AbstractFloat) in Base at float.jl:538
isfinite(x::Integer) in Base at float.jl:541
isfinite(::Irrational) in Base at irrationals.jl:102
isfinite(x::Real) in Base at float.jl:540
isfinite(z::Complex) in Base at complex.jl:117
isfinite(::Union{T, Type{T}}) where T<:Base.Dates.TimeType in Base.Dates at dates/types.jl:316
isfinite(x::AbstractArray{T,N} where N) where T<:Number in Base at deprecated.jl:56

In [6]:
# The following is undefined, but we can define our own version of it.
# +(1, "1")

# Add new method, need to import this
importall Base.Operators

# Add new method
+(x::Number, y::String) = x + parse(Int, y)

+(1, "1")

2

## Type Hierarchy

There are *concrete types* like Int64 and Float32 and *abstract types* like Int, Real, Number, and Any (all types are subtypes of "Any").

In [7]:
# true
Int64 <: Int <: Real <: Number <: Any

# true
Complex64 <: Complex <: Number <: Any

# false
Complex64 <: Complex <: Real

false

In [8]:
Int64 <: Int <: Integer <: Real <: Number <: Any

true

In [9]:
function f(x::Any)
    println("Generic method invoked.")
end

function f(x::Number)
    println("Number function invoked.")
end

function f(x::Integer)
    println("Integer function invoked.")
end

for x in ["cat", -1.1, 1]
    f(x)
end

Generic method invoked.
Number function invoked.
Integer function invoked.


## User-Defined Types

### Default Constructors and Methods

In [10]:
struct Foo
end

foofunc(x::Foo) = "a foo"

# importall Base.Operators
+(x::Foo, y::Foo) = "two foos"

foo = Foo()
hoo = Foo()
typeof(foo)

foo + hoo

"two foos"

In [11]:
mutable struct AR1
    a
    b
    σ
    ϕ
end

using Distributions

m = AR1(0.9, 1, 1, Beta(5, 5))

typeof(m.ϕ)
# mutate ϕ:
m.ϕ = Normal(0, 1)

Distributions.Normal{Float64}(μ=0.0, σ=1.0)

In [12]:
# immutable struct with defined types
# Could also use type Real, but abtract types
# are slower than concrete types.
struct AR1_explicit
    a::Float64
    b::Float64
    σ::Float64
    ϕ::Distribution
end

m = AR1_explicit(0.9, 1, 1, Beta(5, 5))

# Error: immutable
# m.ϕ = Normal()

# Error: wrong type
# m.a = "foo"

AR1_explicit(0.9, 1.0, 1.0, Distributions.Beta{Float64}(α=5.0, β=5.0))

## Type Parameters

In [13]:
# Curly braces { } show parameterized types

# Parameter: Int64
typeof(1 + 2im)

# Parameter: Float64
typeof([1., 2, 3])

Array{Float64,1}

### AR Process

$X_{t+1} = aX_{t} + b + \sigma W_{t+1}$

In [15]:
struct AR1_best{T <: Real}
    a::T
    b::T
    σ::T
    ϕ::Distribution
end

m = AR1_best(0.9, 1., 1., Beta(5, 5))

typeof(m)

AR1_best{Float64}

# Exercises

## Exercise 1

In [73]:
function simulate(m::AR1_best, n::Integer, x0::Real)
    X = Array{Float64}(n+1)
    X[1] = x0
    for t in 1:n
        X[t+1] = m.a * X[t] + m.b + m.σ * rand(m.ϕ)
    end
    return X
end

simulate(m, 10, rand())

11-element Array{Float64,1}:
  0.944362
  2.33129 
  3.67813 
  4.85762 
  5.87232 
  6.72859 
  7.40329 
  8.25971 
  8.88714 
  9.67843 
 10.0613  